# Making predictions using pre-trained networks

In [2]:
# Importing the keras application models
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input,decode_predictions

import numpy as np
!unzip images


Archive:  images.zip
   creating: images/cats/
  inflating: images/cats/cat.1.jpg   
  inflating: images/cats/cat.10.jpg  
  inflating: images/cats/cat.100.jpg  
  inflating: images/cats/cat.101.jpg  
  inflating: images/cats/cat.102.jpg  
  inflating: images/cats/cat.103.jpg  
  inflating: images/cats/cat.104.jpg  
  inflating: images/cats/cat.105.jpg  
  inflating: images/cats/cat.106.jpg  
  inflating: images/cats/cat.107.jpg  
  inflating: images/cats/cat.108.jpg  
  inflating: images/cats/cat.109.jpg  
  inflating: images/cats/cat.11.jpg  
  inflating: images/cats/cat.110.jpg  
  inflating: images/cats/cat.111.jpg  
  inflating: images/cats/cat.112.jpg  
  inflating: images/cats/cat.113.jpg  
  inflating: images/cats/cat.114.jpg  
  inflating: images/cats/cat.115.jpg  
  inflating: images/cats/cat.116.jpg  
  inflating: images/cats/cat.117.jpg  
  inflating: images/cats/cat.118.jpg  
  inflating: images/cats/cat.119.jpg  
  inflating: images/cats/cat.12.jpg  
  inflating: images/c

In [3]:
# creating the model
model=ResNet50(weights="imagenet")

# Reading the image
image_path="hehe.jpg"
img=image.load_img(image_path,target_size=(224,224))
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)
x=preprocess_input(x)


102973440/102967424 [==============================] - 4s 0us/step


In [4]:
# Predicting
preds=model.predict(x)

print("Predicted : ", decode_predictions(preds,top=3)[0] )

40960/35363 [==================================] - 0s 0us/step
Predicted :  [('n02123159', 'tiger_cat', 0.88120043), ('n02123045', 'tabby', 0.07332815), ('n02127052', 'lynx', 0.025835507)]


In [5]:
# Making predictions using out own trained classifer

In [6]:
import os
folders=os.listdir("images")

In [7]:
print (folders)

['horses', 'cats', 'dogs', 'humans']


In [8]:
image_data = []
labels = []
label_dict = { "cats":0,"dogs":1,"horses" : 2,"humans" :3 } 

In [9]:
from keras.preprocessing import image
for ix in folders:
  path=os.path.join("images", ix)

  for im in os.listdir(path):
    img=image.load_img(os.path.join(path,im),target_size=((224,224)))
    img_array=image.img_to_array(img)
    image_data.append(img_array)
    labels.append(label_dict[ix])



In [10]:
print(path)

images/humans


In [11]:
print(len(image_data),len(labels))

808 808


In [12]:
import random
combined=list(zip(image_data,labels))

In [13]:
random.shuffle(combined)
image_data[:],labels[:]=zip(*combined)

In [14]:
print(labels[:10])

[2, 0, 1, 1, 2, 3, 3, 3, 0, 3]


In [15]:
x_train=np.array(image_data)
y_train=np.array(labels)

In [16]:
print(x_train.shape,y_train.shape)

(808, 224, 224, 3) (808,)


In [17]:
#converting labels to one-hot encoder

from keras.utils import np_utils
y_train=np_utils.to_categorical(labels)

In [18]:
y_train.shape

(808, 4)

In [19]:
# Creating the ResNet50 Model 

In [20]:
from keras.applications.resnet50 import ResNet50

In [21]:
from keras.optimizers import Adam
from keras.layers import *
from keras.models import Model

In [22]:
model=ResNet50(include_top=False,weights="imagenet",input_shape=(224,224,3))

94773248/94765736 [==============================] - 2s 0us/step


In [23]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [24]:
# starting making of the classifer by adding dense layer

In [26]:
av1=GlobalAveragePooling2D()(model.output)
fc1=Dense(256,activation="relu")(av1)
d1=Dropout(0.5)(fc1)
fc2=Dense(4,activation="softmax")(d1)

model_new=Model(inputs = model.input , outputs = fc2)         # Model is a  Keras functional API 
model_new.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
_______________________________________________________________________________________

In [27]:
adam=Adam(lr=0.0003)
model_new.compile(loss="categorical_crossentropy",optimizer=adam,metrics=["accuracy"])


In [28]:
for ix in range(len(model_new.layers)):
  print(ix,model_new.layers[ix])

0 <tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7ff25e639128>
1 <tensorflow.python.keras.layers.convolutional.ZeroPadding2D object at 0x7ff25e639438>
2 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7ff307e53198>
3 <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7ff25efdfac8>
4 <tensorflow.python.keras.layers.core.Activation object at 0x7ff25e644f60>
5 <tensorflow.python.keras.layers.convolutional.ZeroPadding2D object at 0x7ff25e644ef0>
6 <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7ff25e5fcb00>
7 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7ff25e607470>
8 <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7ff25e5a5278>
9 <tensorflow.python.keras.layers.core.Activation object at 0x7ff25e5a5f98>
10 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7ff25e5a90b8>
11 <tensorflow.python.keras.layers.normalization_v2.BatchNormalization o

In [29]:
# Making some layers non-trainable
for ix in range(169):
  model_new.layers[ix].trainable=False

In [30]:
# Compiling the model again
model_new.compile(loss="categorical_crossentropy",optimizer=adam,metrics=["accuracy"])

In [31]:
hist=model_new.fit(x_train,y_train,shuffle=True,batch_size=16,epochs=10,validation_split=0.2)

Epoch 1/10
41/41 [==============================] - 3s 69ms/step - loss: 0.4890 - accuracy: 0.8204 - val_loss: 0.1713 - val_accuracy: 0.9198
Epoch 2/10
41/41 [==============================] - 2s 43ms/step - loss: 0.0761 - accuracy: 0.9737 - val_loss: 0.1127 - val_accuracy: 0.9568
Epoch 3/10
41/41 [==============================] - 2s 43ms/step - loss: 0.0373 - accuracy: 0.9876 - val_loss: 0.1029 - val_accuracy: 0.9691
Epoch 4/10
41/41 [==============================] - 2s 44ms/step - loss: 0.0328 - accuracy: 0.9892 - val_loss: 0.1046 - val_accuracy: 0.9691
Epoch 5/10
41/41 [==============================] - 2s 44ms/step - loss: 0.0218 - accuracy: 0.9923 - val_loss: 0.1172 - val_accuracy: 0.9691
Epoch 6/10
41/41 [==============================] - 2s 43ms/step - loss: 0.0097 - accuracy: 1.0000 - val_loss: 0.1244 - val_accuracy: 0.9691
Epoch 7/10
41/41 [==============================] - 2s 43ms/step - loss: 0.0071 - accuracy: 1.0000 - val_loss: 0.1401 - val_accuracy: 0.9630
Epoch 8/10
41